# Visualization

Outline

+ `matplotlib` plotting with lines, markers
+ `astropy.visualization` tools for scaling and stretching images
+ `astrowidgets` for interactive image display
+ `jdaviz` for interactive spectrum display (maybe)

In [ ]:
import numpy as np

# Matplotlib for astro

We will assume going into this that you have at least a bit of familiarity with [matplotlib](https://matplotlib.org/). But in case that's wrong, you will gain a lot by looking through it's docs, and in particular, the [example gallery](https://matplotlib.org/stable/gallery/index.html).

But instead of basics of matplotlib we will focus on ways in which matplotlib and Astropy play well together. Most of the functionality disccussed here is more completely covered in the  [astropy.visualization docs](https://docs.astropy.org/en/stable/visualization/index.html) - this is just a taste.


Lets start with the basics of plotting support for a fairly fundamental thing: `Quantity` objects.

In [ ]:
from astropy import units as u
from astropy import time, coordinates, visualization
from astropy.io import fits

from matplotlib import pyplot as plt

Consider the problem of plotting some sort of fundamentally physical quantity.  For the sake of example, maybe you want to compare the pixel scale of various imagers relative to the primary mirror of their telesope. We can just pull numbers from some random places but if we just write down the numbers without reading carefully:

In [ ]:
pixel_scale = [
   30., #HST ACS as listed in  https://arxiv.org/abs/astro-ph/0703095
   0.031, # JWST NIRCam short from https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-observing-modes/nircam-imaging
   0.17, # Subaru HSC from https://subarutelescope.org//Observing/Instruments/HSC/parameters.html
]

mirror_diameter = [
    2.4, #HST
    6.5, #JWST
    8.2, #Subaru
]

plt.scatter(mirror_diameter, pixel_scale)
plt.xlabel('mirror diameter')
plt.ylabel('pixel scale'); 
# note the semicolon - that just cleans up printing in the notebook to not show a meaningless object in the notebook
# not strictly necessary if you like Python due to the lack of semicolons ;)

But this is utter nonsense, because the ACS pixel scale was quoted in *milli*arcseconds per pixel while the others are arcsec per pixel... This is the sort of confusion quantities can help with:

In [ ]:
pixel_scale = [
   30.*u.milliarcsecond/u.pixel, #HST ACS as listed in  https://arxiv.org/abs/astro-ph/0703095
   0.031*u.arcsecond/u.pixel, # JWST NIRCam short from https://jwst-docs.stsci.edu/jwst-near-infrared-camera/nircam-observing-modes/nircam-imaging
   0.17*u.arcsecond/u.pixel, # Subaru HSC from https://subarutelescope.org//Observing/Instruments/HSC/parameters.html
]

mirror_diameter = [
    2.4*u.m, #HST
    6.5*u.m, #JWST
    8.2*u.m, #Subaru
]

plt.scatter(u.Quantity(mirror_diameter), u.Quantity(pixel_scale))
plt.xlabel('mirror diameter')
plt.ylabel('pixel scale'); 

Ok that's cleaner but it's still easy to get confused why these numbers are so big... thus comes `quantity_support`!

In [ ]:
from astropy.visualization import quantity_support
quantity_support() 

plt.scatter(u.Quantity(mirror_diameter), u.Quantity(pixel_scale))

Now you get units right away that show how the conversion happened.  It will also currently handle *different* units:

In [ ]:
# note we don't have to call quantity_support again because it only needs to get set up once per notebook

ground_based_pixel_scales = [0.17]*u.arcsecond/u.pixel
ground_based_mirror_diameters = [8.2]*u.m

space_based_pixel_scales = [30, 31]*u.milliarcsecond/u.pixel
space_based_mirror_diameters = [2.4, 6.5]*u.m

plt.scatter(ground_based_mirror_diameters, ground_based_pixel_scales)
plt.scatter(space_based_mirror_diameters, space_based_pixel_scales);

Similar situations arise with time.  When you google plotting dates on the internet with matplotlib you frequently encounter plot_date:

In [ ]:
import datetime

In [ ]:
dates = [datetime.datetime(2024, 4, 8),
         datetime.datetime(2020, 1, 20),
         datetime.datetime(2017, 8, 21)
        ]
eclipse = [True, False, True]

plt.plot_date(dates, eclipse);

But if you read the warning you don't need to do this anymore, you can just do this:

In [ ]:
plt.scatter(dates, eclipse);

That said, there are many reasons why you'd rather use astropy times out-of-the-box... But they don't plot as simply:

In [ ]:
astropy_times = time.Time(['2024-4-8', '2020-1-20', '2017-8-21'])

plt.scatter(astropy_times, eclipse);

Of course you can ask for a specific kind of time:

In [ ]:
astropy_times = time.Time(['2024-4-8', '2020-1-20', '2017-8-21'])

plt.scatter(astropy_times.mjd, eclipse);

But you can also just use the similar `time_support`

In [ ]:
from astropy.visualization import time_support
time_support()

plt.scatter(astropy_times, eclipse);

## WCSAxes

`astropy.visualization` includes a few more concrete matplotlib-based primatives as well.  Most notable of these is `WCSAxes`, which does as the name implies: lets you use a WCS from a file or that you created yourself with matplotlib to more easily plot.  This really involves two distinct but closely related operations: plotting images with wcs and plotting other things over those images.  For the purposes of this example we will use an example dataset astropy provides:

In [ ]:
from astropy.utils.data import get_pkg_data_filename

example_image_filename = get_pkg_data_filename('galactic_center/gc_msx_e.fits', 'astropy.visualization')
example_image = fits.open(example_image_filename)
plt.imshow(example_image[0].data, vmin=-2.e-5, vmax=2.e-4)

This looks like some sort of astronomy image... but does not really tell us where it is on the sky.  That's where `WCSAxes` comes in:

In [ ]:
from astropy.wcs import WCS

ax = plt.subplot(projection=WCS(example_image[0].header))
ax.imshow(example_image[0].data, vmin=-2.e-5, vmax=2.e-4);

And with just a bit of care (but no messy fiddling with WCS keywords directly) we can even overplot the coordinate grids even if they aren't align with the pixel grid:

In [ ]:
ax = plt.subplot(projection=WCS(example_image[0].header))
ax.imshow(example_image[0].data, vmin=-2.e-5, vmax=2.e-4)

overlay = ax.get_coords_overlay('fk5')
overlay.grid(color='white', ls='dotted')
overlay[0].set_axislabel('Right Ascension (J2000)')
overlay[1].set_axislabel('Declination (J2000)')

Now lets suppose we want to overplot a coordinate - this is just as straightforward using special plotting functions `plot_coord` and `scatter_coord`:

In [ ]:
coord = coordinates.SkyCoord(266.78238*u.deg, -28.769255*u.deg, frame='fk5')


ax = plt.subplot(projection=WCS(example_image[0].header))
ax.imshow(example_image[0].data, vmin=-2.e-5, vmax=2.e-4)

ax.scatter_coord(coord, color='red');

Even more usefully this works just as well if you have your coordinate in some completely different system than the WCS:

In [ ]:
coord_gal = coordinates.SkyCoord(l=0.31437982*u.deg, b=-0.19565601*u.deg, frame='galactic')

ax = plt.subplot(projection=WCS(example_image[0].header))
ax.imshow(example_image[0].data, vmin=-2.e-5, vmax=2.e-4)

ax.scatter_coord(coord_gal, color='red');

## Image stretching and colorizing

Lastly, astropy.visualization has some helpful machinery to make matplotlib (or other) images stretch in ways astronomer users expect.  To demonstrate the point of this, note that we hand-chose the bounds for the example image above.  What do we get without this?:

In [ ]:
plt.imshow(example_image[0].data)
plt.colorbar();

Ick, most of the information is gone.  But picking `vmin`/`vmax` by hand is also not great because it's very manual.  So instead `astropy.visualization` provides some tools to do this stretching for you.

In [ ]:
perc99 = visualization.PercentileInterval(99)
plt.imshow(perc99(example_image[0].data))
plt.colorbar();

Under the hood this just remapped everything onto 0-1 but clamps out the outermost edges. Similar tools can also be used to adjust the "stretch" of an image:

In [ ]:
perc99 = visualization.PercentileInterval(99)
logstretch = visualization.LogStretch()
plt.imshow(logstretch(perc99(example_image[0].data)))
plt.colorbar();

While for some use cases this is too much fine detail, for some that's exactly the detail you want!

These can be used as-is in whatever visualization technique is desired.  But astropy also provides a convenient wrapper to use this with `imshow`, that helpfully then also better preserves the colorbar information:

In [ ]:
res = visualization.imshow_norm(example_image[0].data, 
                         interval=visualization.PercentileInterval(99),
                         stretch=visualization.LogStretch())
plt.colorbar(res[0], ax=plt.gca());

### RGB images

Lastly, `astropy.visualization` provides a standard function to do a RGB color rendering of a three-band image in the standard method used by the SDSS to preserve color information without distorting luminosity too much.

In [ ]:
g_name = get_pkg_data_filename('visualization/reprojected_sdss_g.fits.bz2', package='astropy.visualization')
r_name = get_pkg_data_filename('visualization/reprojected_sdss_r.fits.bz2', package='astropy.visualization')
i_name = get_pkg_data_filename('visualization/reprojected_sdss_i.fits.bz2', package='astropy.visualization')

g_imagedata = fits.getdata(g_name)
r_imagedata = fits.getdata(r_name)
i_imagedata = fits.getdata(i_name)

First lets look at one of these the way we did before:

In [ ]:
res = visualization.imshow_norm(r_imagedata, 
                         interval=visualization.PercentileInterval(99),
                         stretch=visualization.LogStretch())

That provides a lot of detailed spatial information but can't show the color.  But we can get this with `make_lupton_rgb`:

In [ ]:
rgb_default = visualization.make_lupton_rgb(i_imagedata, r_imagedata, g_imagedata)
plt.imshow(rgb_default, origin='lower')

Now we have a nice RGB image that contains all the color information.  You can experiment with the various parameters of `make_lupton_rgb` (``Q`` and ``stretch``, or stretch/interval objects as shown above) and try to get both good colors and good low-surface brightness details.

# Imviz as an example of a more fully-featured notebook viz tool

The world of visualization is bigger than just these tools, though.  Image visualization in particularly a thriving space - you have likely heard of ds9 and perhaps Ginga.  But there are other tools that are specifically designed for notebook work - e.g. the `jdaviz` tool (where the J has a "strategic ambiguity"...) As a quick demo:

In [ ]:
from jdaviz import Imviz

In [ ]:
imviz = Imviz()

In [ ]:
imviz.show('sidecar')  # this will only work in Jupyterlab, but you can juse just ``imviz.show()`` in more limited environments

In [ ]:
imviz.load_data(example_image[0])

In [ ]:
imviz.default_viewer.zoom_level = 5

In [ ]:
imviz.default_viewer.center_on(coord)

We can explore various UI elements but the keypoint is that they are all available either from the notebook or the UI.

# Astrowidgets: an idea

The catch is: the poor user is then forced to learn a different API if they already like [ginga | pyds9 | js9 | ipyaladin]. 

Enter the astrowidgets concept.  The idea is a standard API that all the tools can use the same:

In [ ]:
import astrowidgets

aw_ginga = astrowidgets.ImageWidget()
aw_ginga

In [ ]:
aw_ginga.load_fits(example_image[0])

In [ ]:
aw_ginga.zoom_level = 5

In [ ]:
aw_ginga.center_on(coord)  

The above actually doesn't work due to a bug (a subtle one about interpreting the WCS)!  But it illustrates how this needs to be a collaborative effort of defining the interface.

# Q&A

* What sort of visualization tools are y'all using?
* What sort of visualization tools would you like to see for your own use?
* What sort of visualization tools would you like to see the community use?
* What applications (if any) do you see for the astrowidgets idea?